In [0]:
customers_schema = '''customer_id INT ,customer_name STRING ,date_of_birth DATE ,telephone STRING ,email STRING ,member_since DATE ,created_timestamp TIMESTAMP'''

In [0]:
df = spark.readStream.format("json").schema(customers_schema).load("/Volumes/dea_course_catalog_gizmobox/landing/operational_data/customers_stream/")

In [0]:
display(df)

In [0]:
from pyspark.sql.functions import current_timestamp,col

In [0]:
customers_df = df.withColumn("ingestion_date",current_timestamp()).withColumn("filepath",col("_metadata.file_path"))


In [0]:
streaming_query = (
    customers_df.writeStream.format("delta")
    .outputMode("append")
    .option(
        "checkpointLocation",
        "/Volumes/dea_course_catalog_gizmobox/landing/operational_data/customers_stream/checkpoint",
    )
    .toTable("dea_course_catalog_gizmobox.bronze.customers_stream")
)

In [0]:
%sql
select * from dea_course_catalog_gizmobox.bronze.customers_stream;

In [0]:
streaming_query.stop()